v0-0-1: model v1_0_0  
v0-0-2: model v1_0_1  
v0-0-3: ansemble model v1_0_0, v1_0_1  
v0-0-4: model v1_0_4  
v0-0-5: ansemble model v1_0_0, v1_0_1 with TTA(rotate, shift, scale)  
v0-0-6: model v1_0_0 with TTA(rotate, shift, scale)  
v0-0-7: model v1_0_5  
v0-0-8: model v1_0_0 with opt bias  
v0-0-9: model v1_0_5_1  
v0-0-10: model v1_0_8  
v0-0-11: ansemble model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate)  
v0-0-12: ansemble model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate)   with opt bias  
v0-0-15: ansemble model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate)   with opt bias 2  
v0-0-16: model v1_0_9  
v0-0-17: ansemble model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate)   with opt bias 3  
v0-0-18: model v1_0_11  

v1-0-0: ansemble (model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate)), (model v1_0_11)   with opt bias 3  
v1-0-1: ansemble (model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate)) with opt bias 3, (model v1_0_11 new) with opt bias v1_0_11
v1-0-2: ansemble (model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate))*0.3 with opt bias 3, (model v1_0_11 new)*0.7 with opt bias v1_0_11  
v1-0-3: ansemble (model v1_0_0, v1_0_1, v1_0_5, v1_0_8 with TTA(rotate))*0.7 with opt bias 3, (model v1_0_11 new)*0.3 with opt bias v1_0_11  

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bengaliai-cv19/train.csv
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/train_multi_diacritics.csv
/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/class_map_corrected.csv
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengali-model/model_v1_0_1/training_log.csv
/kaggle/input/bengali-model/model_v1_0_1/checkpoint
/kaggle/input/bengali-model/model_v1_0_9/training_log.csv
/kaggle/input/bengali-model/model_v1_0_9/checkpoint
/kaggle/input/bengali-model/model_v1_0_5_1/training_log.

data_utils

In [2]:
import os
import gc
import numpy as np
import pandas as pd

class Config:
    data_path = '../input/bengaliai-cv19'

def get_image(type_is_train, height=137, width=236, data_idxs=[0, 1, 2, 3]):
    data_type = 'train' if type_is_train else 'test'

    print('read image')
    image_df_list = [pd.read_parquet(os.path.join(Config.data_path, f'{data_type}_image_data_{i}.parquet')) for i in data_idxs]
    images = [df.iloc[:, 1:].values.reshape(-1, height, width) for df in image_df_list]

    del image_df_list
    gc.collect()
    
    images = np.concatenate(images, axis=0)

    print('image shape', images.shape)
    return images

def get_train_label():
    print('read train label')
    path = os.path.join(Config.data_path, 'train.csv')
    df = pd.read_csv(path)
    print('df size ', len(df))
    print(df.head())

    label = df[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values
    return label

image_proc

In [3]:
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
#from tqdm import tqdm

def crop_resize(img0, size=128, pad=16):
    """
    https://www.kaggle.com/iafoss/image-preprocessing-128x128
    """
    HEIGHT = 137
    WIDTH = 236

    def bbox(img):
        rows = np.any(img, axis=1)
        cols = np.any(img, axis=0)
        rmin, rmax = np.where(rows)[0][[0, -1]]
        cmin, cmax = np.where(cols)[0][[0, -1]]
        return rmin, rmax, cmin, cmax

    # inverse
    img = 255 - img0
    # normalize
    img = (img * 255.0 / img.max()).astype(np.uint8)

    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')

    # resize
    img = cv2.resize(img,(size,size))

    return img

def add_gray_channel(imgs):
    return imgs[:,:,:,None]

class PreprocPipeline:
    def __init__(self):
        self.pl_name = 'pl_base'
        self.data_dir = '../input/train_data'
        self.image_path = os.path.join(self.data_dir, self.pl_name + '_images.npy')

        return

    def save_imgs(self, imgs):
        print('save ', self.image_path)
        np.save(self.image_path, imgs)
        return

    def load_imgs(self):
        print('load ', self.image_path)
        imgs = np.load(self.image_path)
        print('images shape ', imgs.shape)
        return imgs

    def apply_all_image(self, imgs, func):
        pp_imgs = []
        for img in tqdm(imgs):
            pp_imgs.append(func(img))
        pp_imgs = np.array(pp_imgs)
        return pp_imgs

class PreprocPipeline_v1(PreprocPipeline):
    def __init__(self):
        super(PreprocPipeline_v1, self).__init__()
        self.pl_name = 'pl_v1'
        self.data_dir = '../input/train_data'
        self.image_path = os.path.join(self.data_dir, self.pl_name + '_images.npy')

        return

    def preprocessing(self, imgs):
        pp_imgs = self.apply_all_image(imgs, crop_resize)
        pp_imgs = add_gray_channel(pp_imgs)

        return pp_imgs
    
class PreprocPipeline_v5(PreprocPipeline):
    def __init__(self):
        super(PreprocPipeline_v5, self).__init__()
        self.pl_name = 'pl_v5'
        self.data_dir = '../input/train_data'
        self.image_path = os.path.join(self.data_dir, self.pl_name + '_images.npy')

        return

    def preprocessing(self, imgs):
        pp_imgs = add_gray_channel(imgs)

        return pp_imgs

model

In [4]:
import os
import numpy as np
import pandas as pd

import torch
import albumentations as alb
from albumentations.augmentations import transforms as albtr
from albumentations.pytorch import ToTensor as albToTensor

import senet
import modi_senet
import bengali_classifier

In [5]:
def load_model_v1_0_0():
    """
    mish, mixup, cutmix, 
    SENetEncoder_Multiscale_v2+ClassifierModule_v9, 
    dropblock,

    MultilabelStratifiedKFold, 
    class balance(0.999), 
    """
    # checkpoint
    path = '../input/bengali-model/model_v1_0_0/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)
    
    # data
    ts_transformer = alb.Compose([
                        albtr.Normalize(0.5, 0.5),
                        albToTensor()
                        ])
    
    ## model
    get_senet = senet.se_resnext50_32x4d
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 1.0
    mix_cand_layers=[0, 1, 2] #[0, 1, 2, 3]
    cutmix_alpha = None #1.0
    cutmix_cand_layers = None #[0]
    output_layers = [2, 3, 4]
    dropblock_p = 0.2
    encoder = modi_senet.SENetEncoder_CalibMixup_Multiscale_v2(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            output_layers=output_layers, dropblock_p=dropblock_p,
                                            )
    encoded_planes = [512, 1024, 2048] #4 * 512

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v9(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    model.load_state_dict(CP['state_dict'])
    
    return model, ts_transformer

In [6]:
def load_model_v1_0_1():
    """
    mish, mixup, cutmix, 
    SENetEncoder_Multiscale_v2+ClassifierModule_v9, 
    dropblock,

    MultilabelStratifiedKFold, 
    class balance(0.999), 
    """
    # checkpoint
    path = '../input/bengali-model/model_v1_0_1/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)
    
    # data
    ts_transformer = alb.Compose([
                        albtr.Normalize(0.5, 0.5),
                        albToTensor()
                        ])
    
    ## model
    get_senet = senet.se_resnext50_32x4d
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 1.0
    mix_cand_layers=[0, 1, 2] #[0, 1, 2, 3]
    cutmix_alpha = None #1.0
    cutmix_cand_layers = None #[0]
    output_layers = [2, 3, 4]
    dropblock_p = 0.2
    encoder = modi_senet.SENetEncoder_CalibMixup_Multiscale_v2(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            output_layers=output_layers, dropblock_p=dropblock_p,
                                            )
    encoded_planes = [512, 1024, 2048] #4 * 512

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v9(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    model.load_state_dict(CP['state_dict'])
    
    return model, ts_transformer

In [7]:
def load_model_v1_0_4():
    """
    mish, mixup, cutmix, 
    SENetEncoder_Multiscale_v2+ClassifierModule_v9, 
    dropblock,

    MultilabelStratifiedKFold, 
    class balance(0.999), 
    """
    # checkpoint
    path = '../input/bengali-model/model_v1_0_4/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)
    
    # data
    ts_transformer = alb.Compose([
                        albtr.Normalize(0.5, 0.5),
                        albToTensor()
                        ])
    
    ## model
    get_senet = senet.se_resnext50_32x4d
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 1.0
    mix_cand_layers=[0, 1, 2] #[0, 1, 2, 3]
    cutmix_alpha = 1.0
    cutmix_cand_layers = [0, 1, 2, 3]
    encoder = modi_senet.SENetEncoder_Mixup(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            )
    encoded_planes = 2048

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v1(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    model.load_state_dict(CP['state_dict'])
    
    return model, ts_transformer

In [8]:
def load_model_v1_0_5():
    """
    mish, mixup, cutmix, 
    SENetEncoder_Multiscale_v2+ClassifierModule_v9, 
    dropblock,

    MultilabelStratifiedKFold, 
    class balance(0.999), 
    """
    path = '../input/bengali-model/model_v1_0_5/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)

    ## data
    ts_transformer = alb.Compose([
                            albtr.Normalize(0.5, 0.5),
                            albToTensor()
                            ])

    ## model
    get_senet = senet.se_resnext50_32x4d #senet.se_resnet152, senet.se_resnext50_32x4d, senet.se_resnext101_32x4d, senet.se_resnet50
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 1.0
    mix_cand_layers=[0, 1, 2, 3] #[0, 1, 2, 3]
    cutmix_alpha = None #1.0
    cutmix_cand_layers = None #[0]
    output_layers = [4] #[2, 3, 4]
    dropblock_p = 0.2
    encoder = modi_senet.SENetEncoder_CalibMixup_Multiscale_v2(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            output_layers=output_layers, dropblock_p=dropblock_p,
                                            )
    encoded_planes = [2048] #[512, 1024, 2048] #4 * 512

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v9(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    if CP is not None:
        model.load_state_dict(CP['state_dict'])
        
    return model, ts_transformer

In [9]:
def load_model_v1_0_5_1():
    """
    mish, mixup, cutmix, 
    SENetEncoder_Multiscale_v2+ClassifierModule_v9, 
    dropblock,

    MultilabelStratifiedKFold, 
    class balance(0.999), 
    """
    path = '../input/bengali-model/model_v1_0_5_1/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)

    ## data
    ts_transformer = alb.Compose([
                            albtr.Normalize(0.5, 0.5),
                            albToTensor()
                            ])

    ## model
    get_senet = senet.se_resnext50_32x4d #senet.se_resnet152, senet.se_resnext50_32x4d, senet.se_resnext101_32x4d, senet.se_resnet50
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 1.0
    mix_cand_layers=[0] #[0, 1, 2, 3]
    cutmix_alpha = None #1.0
    cutmix_cand_layers = None #[0]
    output_layers = [4] #[2, 3, 4]
    dropblock_p = 0.2
    encoder = modi_senet.SENetEncoder_CalibMixup_Multiscale_v2(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            output_layers=output_layers, dropblock_p=dropblock_p,
                                            )
    encoded_planes = [2048] #[512, 1024, 2048] #4 * 512

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v9(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    if CP is not None:
        model.load_state_dict(CP['state_dict'])
        
    return model, ts_transformer

In [10]:
def load_model_v1_0_8():
    """
    mish, mixup, cutmix, 
    upsampling
    dropblock,

    MultilabelStratifiedKFold, 
    class balance(0.999), 
    """
    path = '../input/bengali-model/model_v1_0_8/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)

    ## data
    ts_transformer = alb.Compose([
                            albtr.Normalize(0.5, 0.5),
                            albToTensor()
                            ])

    ## model
    get_senet = senet.se_resnext50_32x4d #senet.se_resnet152, senet.se_resnext50_32x4d, senet.se_resnext101_32x4d, senet.se_resnet50
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 1.0
    mix_cand_layers=[0]
    cutmix_alpha = None #1.0
    cutmix_cand_layers = None #[0, 1, 2, 3]
    #output_layers = [3, 4] #[2, 3, 4]
    dropblock_p = 0.2
    upsample_size = None
    calib_mixup = True
    encoder = modi_senet.SENetEncoder_Mixup(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            dropblock_p=dropblock_p, upsample_size=upsample_size, calib_mixup=calib_mixup,
                                            )
    encoded_planes = 2048 #[1024, 2048] #[512, 1024, 2048] #4 * 512

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v1(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    if CP is not None:
        model.load_state_dict(CP['state_dict'])

    return model, ts_transformer

In [11]:
def load_model_v1_0_9():
    """
    mish, mixup, cutmix, 
    upsampling
    dropblock,

    MultilabelStratifiedKFold, 
    class balance(0.999), 
    """
    path = '../input/bengali-model/model_v1_0_9/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)

    ## data
    ts_transformer = alb.Compose([
                            albtr.Normalize(0.5, 0.5),
                            albToTensor()
                            ])

    ## model
    get_senet = senet.se_resnext101_32x4d #senet.se_resnet152, senet.se_resnext50_32x4d, senet.se_resnext101_32x4d, senet.se_resnet50
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 0.2
    mix_cand_layers=[0, 1, 2, 3]
    cutmix_alpha = None #1.0
    cutmix_cand_layers = None #[0, 1, 2, 3]
    #output_layers = [3, 4] #[2, 3, 4]
    dropblock_p = 0.2
    upsample_size = None
    calib_mixup = False
    encoder = modi_senet.SENetEncoder_Mixup(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            dropblock_p=dropblock_p, upsample_size=upsample_size, calib_mixup=calib_mixup,
                                            )
    encoded_planes = 2048 #[1024, 2048] #[512, 1024, 2048] #4 * 512

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v1(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    if CP is not None:
        model.load_state_dict(CP['state_dict'])

    return model, ts_transformer

In [12]:
def load_model_v1_0_11():
    """
    PreprocPipeline_v5
    mish, mixup, cutmix, 
    SENetEncoder_Multiscale_v2+ClassifierModule_v9, 
    dropblock,

    class balance(0.999),  
    """
    # checkpoint
    path = '../input/bengali-model/model_v1_0_11/checkpoint'
    CP = torch.load(path, map_location=lambda storage, loc: storage)
    
    # data
    ts_transformer = alb.Compose([
                        albtr.Normalize(0.5, 0.5),
                        albToTensor()
                        ])
    
    ## model
    get_senet = senet.se_resnext50_32x4d
    input3ch = False
    three_neck = False
    use_mish = True
    mixup_alpha = 1.0
    mix_cand_layers=[0, 1, 2] #[0, 1, 2, 3]
    cutmix_alpha = None #1.0
    cutmix_cand_layers = None #[0]
    output_layers = [2, 3, 4]
    dropblock_p = 0.2
    encoder = modi_senet.SENetEncoder_CalibMixup_Multiscale_v2(get_senet(num_classes=1000, pretrained=None), 
                                            input3ch=input3ch,
                                            three_neck=three_neck, mixup_alpha=mixup_alpha, mix_cand_layers=mix_cand_layers,
                                            use_mish=use_mish, cutmix_alpha=cutmix_alpha, cutmix_cand_layers=cutmix_cand_layers, 
                                            output_layers=output_layers, dropblock_p=dropblock_p,
                                            )
    encoded_planes = [512, 1024, 2048] #4 * 512

    dropout_p = 0.1
    classifier = bengali_classifier.ClassifierModule_v9(encoded_planes, dropout_p=dropout_p)

    encoder_use_mixup = True
    model = bengali_classifier.BengaliClassifier_v2(encoder, classifier, 
                                                    encoder_use_mixup, 
                                                    )
    model.load_state_dict(CP['state_dict'])
    
    return model, ts_transformer

opt bias

In [13]:
def get_opt_bias_v1_0_0():
    _gra_bias = np.array([[-0.480095469,-0.016881483,-0.711119766,0.10850629,-0.319399709,0.069398622,-0.135471127,0.094692046,-0.22350041,-0.579261425,0.262861927,-0.985228343,-0.139366013,-0.248861043,0.813733566,0.115876421,0.072112085,0.302518343,0.011260215,1.008772543,0.143659503,0.63032142,-0.54605066,-0.737825211,0.283757345,-0.072300979,-0.303228707,0.003254894,0.728028259,-0.298086694,-0.097845685,-0.022713758,-0.096610968,0.421955963,0.570242236,0.093728298,0.941389599,0.080140348,-0.572876715,0.453637305,-0.130356827,-0.209279951,0.106894913,-0.743732772,0.379835594,-0.033942808,0.204503796,-0.197780913,0.230638457,0.435639251,0.392954555,0.409499784,0.698014418,-0.730208664,-0.185532613,-0.766582936,-0.590030427,0.650983753,-0.737300488,-1.203949302,-0.048263047,0.194522807,0.409832225,-1.044175653,-0.569670989,0.065457351,0.005610182,0.609698001,0.852427416,-0.170532615,-0.118382993,-1.254647542,-1.211620198,0.518037736,-0.640135098,-0.092230476,0.37586223,-0.159990335,-0.121556049,-0.593528856,0.123320761,-0.733383744,-0.141545881,-0.322351751,-0.506350018,0.107385156,0.216412832,0.681361075,0.693468841,-0.110146762,0.150434332,-0.44401259,-0.799180406,-0.1899018,0.518858037,0.565720217,0.015732221,-0.316799629,0.033470814,0.318641702,-0.254744523,0.366212791,-0.765336627,-0.064608276,-0.752780982,0.183698046,0.719176798,-0.5918576,0.925261434,-0.275785888,0.803314172,0.255969875,0.245653772,0.028978994,-0.777718048,-0.499860879,0.361035731,0.759730263,-0.065735175,0.2424246,0.135957369,0.687041309,-0.763946306,-0.568317549,-0.436862959,0.607176903,0.352472314,0.471452067,0.208956392,0.279108628,-0.59115568,0.387108045,-0.004316348,-0.032804359,-0.090443649,0.148990943,-0.359227119,-0.01178028,0.649877548,-0.077646952,0.617578208,1.061098425,-0.069097379,-0.171722473,-0.080523649,0.721767048,-0.182238928,-0.444286081,-0.482474551,0.230594311,-0.225021457,0.399295433,0.52823926,0.064587626,0.027128103,0.403198709,0.132751488,1.008361671,-0.511369121,-0.276320658,0.275967049,-0.478348356,0.599504041,0.012541137,-0.263628204,0.119695416,-0.649403461,0.081498586]])
    _vow_bias = np.array([[-0.820585915,-0.616178567,0.166204854,-0.124035898,-0.118850475,0.624354004,-0.071099554,-0.414545534,0.572863538,-0.049073234,0.802649968]])
    _con_bias = np.array([[-1.115520839,-0.173235148,-0.213842141,0.880537611,-0.597744881,-0.006991053,1.225932067]])
    return _gra_bias, _vow_bias, _con_bias

In [14]:
def get_opt_bias_ensemble1():
    print('get_opt_bias_ensemble1')
    _gra_bias = np.array([[-0.0765897325220675,-0.419975792526503,0.0467195362737845,-0.409788526670828,0.48485361682875,-0.562631575864829,0.168333031039506,0.660516233568517,-0.676709064157098,-0.654681249903989,0.417754618185793,-0.49016010357714,-0.732480747000268,-0.345472845603314,-0.0230093574755144,0.346055389599492,0.66338336467635,0.319139953536027,-0.655485545430353,1.02607500955444,0.924178225506031,-0.0296057549487161,-0.450868720577867,-1.01599036080794,-0.570269600076262,-0.199312817017034,-0.26246866769891,1.13443653157708,1.04793427418054,-0.639889463549989,-0.139944291988522,0.624334468357219,0.450546932541054,-0.013381153055307,-0.397330518122654,0.238165073422953,0.391709965361372,-0.349361943196798,-0.208564758027208,0.0387994002179555,-0.394257078488215,-0.0281522277602335,0.229501310253296,0.0149069837516725,-0.109728043885381,-0.287840996867732,-0.0829268750074971,-1.11020674894836,-0.016528113863467,-0.668153381797915,-0.12150265406308,-0.179970692968879,-0.00683669262442924,0.138782464584288,1.38426553872163,0.0679932450285385,-0.425014183060125,0.0407322345001001,-0.0618676938063015,-0.726867318201559,0.356270440166542,0.316258299933508,0.748221502461295,-0.233006162296327,-0.40500129346491,-0.305568320355354,0.0348603597647733,0.344753872139729,0.709654216104489,0.0341569660767937,-0.193719187486059,-1.31647530484027,-0.713936672126039,0.235624479059272,-0.336353087310415,-0.488035291423839,0.349433977893149,-0.145294149139832,-0.383724042812487,0.419695702677247,0.325543580432679,-0.784938859564148,-0.0662535721788085,-0.142206069610067,-0.110865181982107,0.269288984397744,-0.027377644978774,0.799258479839416,0.8682188608143,-0.367134796148258,0.588367983898326,-0.109302756583794,-0.606249846990915,0.777877163964047,0.132705570392108,1.04554378762373,-0.665195925508171,-0.300027331681324,-0.23375949221675,0.456544501470025,0.148125626771951,0.871715553905812,-0.21104896194028,-0.51589894303238,-0.0848390693765712,-0.879726729834346,0.532567693118531,-0.358692185509413,0.608865058479404,0.231629994316063,-0.0429862200391737,-0.614924325103964,0.299911202485781,-0.243558575142283,-0.490552257109804,-0.339519008995304,0.772421092696636,-0.165560361660667,0.0729013390151752,0.0292963752314103,-0.110972112036528,1.27984475568041,-0.668531289549961,-0.21824281053127,-0.487490480222428,0.864276998905589,-0.136892139321912,-0.136428410278103,0.116963370716739,0.235588758347203,-0.248783611576053,-0.096358458009889,-0.186713294998844,-0.803308466230601,0.009312471977984,-0.163318620867985,-0.0532531645385332,0.182604429011664,0.188520733584192,0.0252596882340723,0.62166670082445,1.15637932729857,-0.0248689774887115,-0.120348954122123,1.02264219428928,1.12633630715853,-0.568917992342892,-0.243429382710954,-0.668122209584687,-0.263906031006031,-0.0817144011522939,0.921857446636221,-0.0020439078239355,-0.132245294173967,-0.023563570589382,-0.5391666798694,-0.280503724271213,1.13986313424996,-0.0690528413158765,-0.365362229112433,0.63566731582994,-0.0524569672906696,0.0744613839200983,0.244106676023624,-0.301280919566222,-0.419159090203196,-0.0214973029604792,-0.0141997477141653]])
    _vow_bias = np.array([[-0.975653237549388,-0.950608279766789,-0.0877742109775104,-0.0581375967860407,-0.068548306712695,1.13266844720795,0.232868115308781,-0.488622541389067,0.600176113450147,-0.337826774380663,0.996370456101412]])
    _con_bias = np.array([[-1.71631017944982,0.0641022863984445,-0.321387719637813,1.654404336525,-0.1622939966067,-0.380872178067437,0.860640774630417]])
    return _gra_bias, _vow_bias, _con_bias

In [15]:
def get_opt_bias_ensemble2():
    print('get_opt_bias_ensemble2')
    _gra_bias = np.array([[0.160973664580358,-0.135945177957371,0.00619334807564855,-0.288490450153015,0.466283180888504,-0.582519134596592,0.0894342699530346,0.745762179894369,-0.815348542980589,-0.42346033582482,0.292426153732067,-0.484925039563689,-0.606151073425006,-0.359692073005706,0.0984661064147318,0.353012916754404,0.438002289705028,0.378421808589382,-0.55142650681927,1.00901516615399,1.09503686414223,0.0149316363979563,-0.421476267037989,-0.960219740630326,-0.416947123579566,-0.339309657165976,-0.389626520106315,0.994338540768622,0.858649766080398,-0.525468642500989,-0.231125011237571,0.581057104949918,0.395364093459365,-0.215373140923603,-0.647181529706599,0.227761763645055,0.338626710388435,-0.165484753778874,-0.338982821385008,0.237193501591445,-0.441987743647545,-0.352939888490742,0.0718692059633357,-0.0466021473153274,-0.115324652595596,-0.257177389769176,-0.0617056398442256,-1.06281524508007,-0.137407645463096,-0.447475451922935,-0.0454383795155019,-0.0289330108559709,0.0279498833217093,0.0284370169877306,1.28232442156519,-0.0685851981552067,-0.418869382874052,-0.0805558453356332,-0.145812807559688,-0.663065404790993,0.346415808499999,0.314128935295667,0.576329328133903,-0.159607922802722,-0.383269461714128,-0.479092727089165,0.0781563080877262,0.379139279371416,0.594052203791675,0.0458276253369213,-0.232791539679731,-1.30473234647459,-0.759372607434553,-0.164017091631228,-0.440441445192217,-0.521214352029609,0.462152286249934,-0.218931463602955,-0.43323907364815,0.385226178167272,0.314781640996159,-0.781888234646681,-0.0615570099951106,-0.189861629613568,-0.0986814486747467,0.14397435799143,-0.129814333395791,0.975442850883964,0.763081948764536,-0.261611558949931,0.153227379689443,-0.09755734264847,-0.57913643199971,0.854141649106908,0.166087944872797,1.08452348506959,-0.553421615705599,-0.420984962078773,-0.123943073059685,0.509909387006039,0.213155057951675,0.896896351206515,-0.374800732296994,-0.44805808008574,0.0295650614427045,-0.689838996539041,0.681190093092396,-0.600618427374332,0.640758331304732,-0.0581069400116885,-0.0197011254967141,-0.688755203586666,0.16858486841435,-0.362840158871446,-0.24075973672856,-0.400741678707614,0.828684117137666,0.0399577455804463,0.0173885678532421,0.100495716808747,-0.202696249427398,1.35417147070138,-0.600102633153812,-0.239981248804995,-0.331687222009774,0.859108401337161,0.095043617301458,0.0665803970420767,0.225477677104895,0.252458835407249,-0.104592016834626,-0.022022228087023,-0.188962471526418,-0.867037145148049,0.0242845288946305,-0.117806928111356,0.0839807028643468,0.135113710265209,0.122187000213395,0.117675382308727,0.653989844790136,1.14518425917136,-0.150566502002602,-0.151134963763231,1.09793263125593,1.09551386489519,-0.450347500102671,-0.326674739249886,-0.769472121981281,-0.152832249122653,0.16512227998829,0.628072408468806,0.0117484270466963,0.162140341510509,-0.0856155717857297,-0.274038370346974,-0.222540003582129,1.03375896541215,-0.0866278891153836,-0.257529196451887,0.422088763378954,0.131614276584857,0.150693485084362,-0.0854054849717169,-0.353259217729774,-0.284579178465606,0.0404412962584792,0.0494589571661498]])
    _vow_bias = np.array([[-0.995763974967762,-1.0793468334142,-0.0591861059402275,-0.143326411327194,0.0627926195402888,1.03310090563859,0.105048171620568,-0.450584110066836,0.741677481599767,-0.262422847901027,1.05553701990919]])
    _con_bias = np.array([[-1.77534418739495,0.0835870125497223,-0.335418502535937,1.90388629848119,-0.239138969587816,-0.433192635184399,0.797667468676797]])
    return _gra_bias, _vow_bias, _con_bias

In [16]:
def get_opt_bias_v1_0_11():
    print('get_opt_bias_v1_0_11')
    _gra_bias = np.array([[-0.107484865822669,0.282381796547515,-0.0815130283932434,-0.306958717045288,-0.111557624772361,0.625793873900685,-0.149728817446114,0.0473088361441221,-0.431851394455569,-0.787404589686012,-0.154743781083754,-0.191520369264067,0.040506381619831,-0.297343526925909,0.713736434804968,-0.416821337719143,-0.126666684211062,0.782424509556823,-0.892272631258389,1.06159016186518,0.098411187313107,0.431786479199579,-0.004617612985301,-0.0305163893999415,-0.458458611740396,0.332429605810422,-0.086563482375128,0.808878310731006,0.689325356589452,-0.0623852348981255,-0.00338631802919305,0.446611530111997,-0.176737010244269,-0.698279058767481,-0.311580314650485,0.551428990656806,0.805456196416346,-0.209845332030607,-0.573603548872218,-0.642716651878348,-0.728903368798493,0.611348547800167,0.837568549515468,-0.355591127529293,0.491118981849618,-0.00725799642323049,0.380357082131746,-0.415196260793962,-0.202860994841226,0.316317507707928,-0.388974013549623,0.270054088633741,-0.116514336313315,-0.787972253122155,0.63289379407734,-0.155950297320279,-0.765517175475103,-0.034685803776024,0.000372004185211799,-0.060510412759347,0.001538465545479,-0.353841783183981,0.985381178542611,-0.187251841706635,-0.203529780965793,-0.129093356444934,0.317392857555696,0.27029781672084,0.210617403733372,-0.0271742077429495,-0.622117845876423,-0.60061084695556,-0.434823513905907,-0.819603851625275,-0.161973150130949,-0.469917641727516,0.270330789406107,0.434833468235871,-0.921873779351036,-1.02504128001257,-0.382550866138137,-0.712356535750289,-0.452960364704173,0.172338311766133,0.153017626183489,0.270983803824535,0.00939313902121899,0.521737083110975,0.579965355743378,0.247397784256265,-0.625272969944396,-0.785336895885165,-0.167416020021925,0.549169718489714,1.08059991206657,0.10555577897043,-0.299916429512755,1.08394274312512,-0.25171047271581,0.321678289640662,0.318280722558845,0.51539518696656,-0.758993727018123,0.555204789841263,0.209637209740763,0.248951055927762,0.377830832995163,-0.802428783573689,0.303639261708814,-0.333962032924156,-1.10045669637466,-0.463610413140135,-0.45009300037892,-0.239099342423245,-0.212809589358385,-0.611796644752622,0.0988877458688426,-0.479320950432976,-0.133545178082345,0.017554233408371,-0.593578725420756,-0.141175855320048,-0.550135265236871,0.049240040010104,-0.280911754963413,0.345011640128565,0.450949453809104,0.165830725971081,0.131970185694383,0.996613251544773,-0.216582200025139,-0.306970141501688,0.31698830940198,-0.263957069841881,0.218592499770144,0.547269423091362,0.0525178358258335,-0.666342667165339,0.125282090575263,-0.434323819532812,0.602418975751125,-0.467087945397529,-0.000171691573180491,-0.777878120314114,0.276334125163251,1.67494198632043,-0.070918536777692,0.185501497246207,-0.680979968911323,-0.459182351388679,0.40611293212594,-0.0808161611382625,-0.103266742748936,0.517568044411761,0.385013395511501,1.04230086193411,0.0929616556418665,-0.0460771856072545,-0.00711353968142798,0.270649165916138,0.331864778186648,1.30584780235372,0.795855509463266,0.729720351752454,-0.077924764845804,-0.320775123142957,0.225597260847892,-0.247721008001435]])
    _vow_bias = np.array([[-1.35578521925312,-0.696488944455397,-0.263156749323781,-0.510848332096763,-0.138981552265663,1.04841786440289,1.06606834992828,-0.543002583952438,0.866280133469548,-0.0398101895943839,0.571549681178354]])
    _con_bias = np.array([[-0.979591095664484,-0.0451350943230734,-0.169955727365026,0.185047618175944,-0.457393883379323,0.345242074695729,1.11870897766352]])
    return _gra_bias, _vow_bias, _con_bias

data loader

In [17]:
import torch_data_utils as tdu

def get_dataloader(imgs, transformer, batchsize):
    ds = tdu.ImgDataset(imgs, None, transformer)
    dl = tdu.get_dataloader(ds, batchsize, shuffle=False)
    return dl    

predict

In [18]:
def predict_logit(net, loader):
    net = net.cuda()
    net.eval()
    
    gra = []
    vow = []
    con = []
    
    print('predict logit')
    with torch.no_grad():
        for batch_idx, imgs in enumerate(tqdm(loader)):
            imgs = imgs.cuda()

            outputs = net(imgs)
 
            gra.append(outputs[0].cpu().numpy())
            vow.append(outputs[1].cpu().numpy())
            con.append(outputs[2].cpu().numpy())
            
    gra = np.concatenate(gra)
    vow = np.concatenate(vow)
    con = np.concatenate(con)
    
    print('gra', gra.shape)
    print('vow', vow.shape)
    print('con', con.shape)
    
    return gra, vow, con

def predict_label(net, loader):
    net = net.cuda()
    net.eval()
    
    gra = []
    vow = []
    con = []
    
    print('predict logit')
    with torch.no_grad():
        for batch_idx, imgs in enumerate(tqdm(loader)):
            imgs = imgs.cuda()

            outputs = net(imgs)
 
            gra.append(outputs[0].max(1)[1].cpu().numpy())
            vow.append(outputs[1].max(1)[1].cpu().numpy())
            con.append(outputs[2].max(1)[1].cpu().numpy())
            
    gra = np.concatenate(gra)
    vow = np.concatenate(vow)
    con = np.concatenate(con)
    
    print('gra', gra.shape)
    print('vow', vow.shape)
    print('con', con.shape)
    
    return gra, vow, con

In [19]:
def logit_to_index(logit):
    index = np.argmax(logit, axis=1)
    return index

def logit3_to_label(logit3):
    """
    Args: tuple (logit0, logit1, logit2)
    """
    label0 = np.argmax(logit3[0], axis=1)
    label1 = np.argmax(logit3[1], axis=1)
    label2 = np.argmax(logit3[2], axis=1)
    return label0, label1, label2

TTA

In [20]:
import torch
import torch.nn as nn
from albumentations.augmentations import functional as albF

import numpy as np
import cv2

class TTA:
    def __init__(self, net, transform, degrees=[0], scales=[0], shifts=[[0,0]], low_thresh=None):
        self.net = net
        self.transform = transform
        self.low_thresh = low_thresh

        self.tta_args = self.get_tta_args(degrees, scales, shifts)

    def get_tta_args(self, degrees, scales, shifts):
        tta_args = []
        tta_args.append({'degree':0.0, 'scale':0.0, 'shift':[0.0,0.0]})

        for deg in degrees:
            if deg != 0:
                tta_args.append({'degree':deg, 'scale':0.0, 'shift':[0.0,0.0]})

        for sca in scales:
            if sca != 0:
                tta_args.append({'degree':0.0, 'scale':sca, 'shift':[0.0,0.0]})

        for shi in shifts:
            if shi[0] != 0 and shi[1] != 0:
                tta_args.append({'degree':0.0, 'scale':0.0, 'shift':shi})

        return tta_args

    @staticmethod
    def shift_scale_rotate(degree, scale, shift):
        def func(img):
            return albF.shift_scale_rotate(img, degree, 1.0 + scale, shift[0], shift[1], cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
        return func

    def predict(self, images, batch_size):
        preds = None

        for tta_arg in self.tta_args:
            print(tta_arg)
            ds = _ImgDataset(images, self.transform, TTA.shift_scale_rotate(**tta_arg))
            loader = _get_dataloader(ds, batch_size)

            output = predict_logit(self.net, loader)
            if preds is None:
                preds = list(output)
                if self.low_thresh is not None:
                    for i in range(len(preds)):
                        preds[i] = np.maximum(preds[i], self.low_thresh)

            else:
                for i, oup in enumerate(output):
                    if self.low_thresh is None:
                        preds[i] += oup
                    else:
                        preds[i] += np.maximum(oup, self.low_thresh)


        for i in range(len(preds)):
            preds[i] = preds[i] / len(self.tta_args)

        return preds

class _ImgDataset(torch.utils.data.Dataset):
    def __init__(self, imgs, transform=None, add_transform=None):
        self.imgs = imgs
        self.transform = transform
        self.add_transform = add_transform

    def __getitem__(self, idx):
        img = self.imgs[idx]

        if self.add_transform is not None:
            img = self.add_transform(img)

        if self.transform is not None:
            img = self.transform(image=img)['image']

        return img

    def __len__(self):
        return len(self.imgs)

def _get_dataloader(dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

submission

In [21]:
def make_submission(labels):
    """
    Args: labels: tuple (gra, vow, con)
    """
    print('make submission')
    row_id = []
    target = []
    
    n = len(labels[0])
    print('num image ', n)
    
    for i in range(n):
        row_id.append('Test_' + str(i) + '_consonant_diacritic')
        target.append(labels[2][i])
        row_id.append('Test_' + str(i) + '_grapheme_root')
        target.append(labels[0][i])
        row_id.append('Test_' + str(i) + '_vowel_diacritic')
        target.append(labels[1][i])
    
    sub = pd.DataFrame({'row_id': row_id, 'target': target})
    sub.to_csv('submission.csv', index=False)
    
    print(sub.head())
    return

test

In [22]:
def calc_logit(mdl, mdl_tf, imgs, logits, batch_size):
    dataloader = get_dataloader(imgs, mdl_tf, batch_size)

    lgt = predict_logit(mdl, dataloader)
    logits = update_logit(logits, lgt)
    return logits

def update_logit(logits, new_logits):
    if logits is None:
        logits = new_logits
    else:
        logits = list(logits)
        for j in range(len(logits)):
            logits[j] = logits[j] + new_logits[j]
            
    return logits

In [23]:
if False:
    # without TTA

    data_idxs=[0, 1, 2, 3]

    logit0, logit1, logit2 = [], [], []

    for i in data_idxs:
        images = get_image(type_is_train=False, height=137, width=236, data_idxs=[i])
        if i == 0:
            print(images.shape)

        pp_pl = PreprocPipeline_v1()
        images = pp_pl.preprocessing(images)
        if i == 0:
            print(images.shape)

        batch_size = 512
        logits = None
        n_model = 0

        """
        # v1_0_0
        mdl, mdl_tf = load_model_v1_0_0()    
        if i == 0:
            print('v1_0_0')
            print(mdl)
        logits = calc_logit(mdl, mdl_tf, images, logits, batch_size)
        logits = update_logit(logits, get_opt_bias_v1_0_0()) # with opt bias
        n_model += 1
        """
        """
        # v1_0_1
        mdl, mdl_tf = load_model_v1_0_1()
        if i == 0:
            print('v1_0_1')
            print(mdl)
        logits = calc_logit(mdl, mdl_tf, images, logits, batch_size)
        n_model += 1
        """
        """
        # v1_0_5
        mdl, mdl_tf = load_model_v1_0_5()
        if i == 0:
            print('v1_0_5')
            print(mdl)
        logits = calc_logit(mdl, mdl_tf, images, logits, batch_size)
        n_model += 1
        """
        """
        # v1_0_5_1
        mdl, mdl_tf = load_model_v1_0_5_1()
        if i == 0:
            print('v1_0_5_1')
            print(mdl)
        logits = calc_logit(mdl, mdl_tf, images, logits, batch_size)
        n_model += 1
        """
        """
        # v1_0_8
        mdl, mdl_tf = load_model_v1_0_8()
        if i == 0:
            print('v1_0_8')
            print(mdl)
        logits = calc_logit(mdl, mdl_tf, images, logits, batch_size)
        n_model += 1
        """
        """
        # v1_0_9
        mdl, mdl_tf = load_model_v1_0_9()
        if i == 0:
            print('v1_0_9')
            print(mdl)
        logits = calc_logit(mdl, mdl_tf, images, logits, batch_size)
        n_model += 1
        """

        # average
        logits = list(logits)
        for j in range(len(logits)):
            logits[j] = logits[j] / n_model

        logit0.append(logits[0])
        logit1.append(logits[1])    
        logit2.append(logits[2])

    logit0 = np.concatenate(logit0, axis=0)
    logit1 = np.concatenate(logit1, axis=0)
    logit2 = np.concatenate(logit2, axis=0)

    labels = logit3_to_label((logit0, logit1, logit2))

In [24]:
if False:
    # without TTA, image 137x236

    data_idxs=[0, 1, 2, 3]

    logit0, logit1, logit2 = [], [], []

    for i in data_idxs:
        images = get_image(type_is_train=False, height=137, width=236, data_idxs=[i])
        if i == 0:
            print(images.shape)

        pp_pl = PreprocPipeline_v5()
        images = pp_pl.preprocessing(images)
        if i == 0:
            print(images.shape)

        batch_size = 512
        logits = None
        n_model = 0

        # v1_0_11
        mdl, mdl_tf = load_model_v1_0_11()    
        if i == 0:
            print('v1_0_11')
            print(mdl)
        logits = calc_logit(mdl, mdl_tf, images, logits, batch_size)
        logits = update_logit(logits, get_opt_bias_v1_0_0()) # with opt bias
        n_model += 1

        # average
        logits = list(logits)
        for j in range(len(logits)):
            logits[j] = logits[j] / n_model

        logit0.append(logits[0])
        logit1.append(logits[1])    
        logit2.append(logits[2])

    logit0 = np.concatenate(logit0, axis=0)
    logit1 = np.concatenate(logit1, axis=0)
    logit2 = np.concatenate(logit2, axis=0)

    labels = logit3_to_label((logit0, logit1, logit2))

In [25]:
if True:
    # with TTA
    data_idxs=[0, 1, 2, 3]

    logit0, logit1, logit2 = [], [], []

    tta_args = {'degrees':[0, -10, 10], 
                'scales':[0],#[0, 0.1, -0.1],
                'shifts':[[0,0]],#[[0,0], [-0.05,-0.05], [0.05, 0.05]],
               }

    for i in data_idxs:
        images = get_image(type_is_train=False, height=137, width=236, data_idxs=[i])
        if i == 0:
            print(images.shape)

        pp_pl = PreprocPipeline_v1()
        images = pp_pl.preprocessing(images)
        if i == 0:
            print(images.shape)

        batch_size = 512
        logits = None
        n_model = 0


        # v1_0_0
        mdl, mdl_tf = load_model_v1_0_0()    
        if i == 0:
            print('v1_0_0')
            print(mdl)
        tta = TTA(mdl, mdl_tf, **tta_args)
        logits = update_logit(logits, tta.predict(images, batch_size))
        n_model += 1

        # v1_0_1
        mdl, mdl_tf = load_model_v1_0_1()
        if i == 0:
            print('v1_0_1')
            print(mdl)
        tta = TTA(mdl, mdl_tf, **tta_args)
        logits = update_logit(logits, tta.predict(images, batch_size))
        n_model += 1
        
        # v1_0_5
        mdl, mdl_tf = load_model_v1_0_5()
        if i == 0:
            print('v1_0_5')
            print(mdl)
        tta = TTA(mdl, mdl_tf, **tta_args)
        logits = update_logit(logits, tta.predict(images, batch_size))
        n_model += 1
        
        # v1_0_8
        mdl, mdl_tf = load_model_v1_0_8()
        if i == 0:
            print('v1_0_8')
            print(mdl)
        tta = TTA(mdl, mdl_tf, **tta_args)
        logits = update_logit(logits, tta.predict(images, batch_size))
        n_model += 1
        
        # average
        logits = list(logits)
        for j in range(len(logits)):
            logits[j] = logits[j] / n_model

        # opt bias
        #logits = update_logit(logits, get_opt_bias_ensemble1()) # with opt bias
        logits = update_logit(logits, get_opt_bias_ensemble2()) # with opt bias
            
        logit0.append(logits[0])
        logit1.append(logits[1])    
        logit2.append(logits[2])

    logit0 = np.concatenate(logit0, axis=0)
    logit1 = np.concatenate(logit1, axis=0)
    logit2 = np.concatenate(logit2, axis=0)

    labels = logit3_to_label((logit0, logit1, logit2))


read image
image shape (3, 137, 236)
(3, 137, 236)



(3, 128, 128, 1)
v1_0_0
BengaliClassifier_v2(
  (encoder): SENetEncoder_CalibMixup_Multiscale_v2(
    (layer0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True


gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': -10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': 10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
v1_0_1
BengaliClassifier_v2(
  (encoder): SENetEncoder_CalibMixup_Multiscale_v2(
    (layer0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, moment

...


gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': 10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': 0.0, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': -10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': 10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
get_opt_bias_ensemble2


In [26]:
if True:
    # with TTA, image 137x236
    data_idxs=[0, 1, 2, 3]

    logit0_2, logit1_2, logit2_2 = [], [], []

    tta_args = {'degrees':[0, -10, 10], 
                'scales':[0],#[0, 0.1, -0.1],
                'shifts':[[0,0]],#[[0,0], [-0.05,-0.05], [0.05, 0.05]],
               }

    for i in data_idxs:
        images = get_image(type_is_train=False, height=137, width=236, data_idxs=[i])
        if i == 0:
            print(images.shape)

        pp_pl = PreprocPipeline_v5()
        images = pp_pl.preprocessing(images)
        if i == 0:
            print(images.shape)

        batch_size = 512
        logits = None
        n_model = 0
        
        # v1_0_11
        mdl, mdl_tf = load_model_v1_0_11()    
        if i == 0:
            print('v1_0_11')
            print(mdl)
        tta = TTA(mdl, mdl_tf, **tta_args)
        logits = update_logit(logits, tta.predict(images, batch_size))
        n_model += 1
        
        # average
        logits = list(logits)
        for j in range(len(logits)):
            logits[j] = logits[j] / n_model

        # opt bias
        #logits = update_logit(logits, get_opt_bias_ensemble1()) # with opt bias
        logits = update_logit(logits, get_opt_bias_v1_0_11()) # with opt bias
            
        logit0_2.append(logits[0])
        logit1_2.append(logits[1])    
        logit2_2.append(logits[2])

    logit0_2 = np.concatenate(logit0_2, axis=0)
    logit1_2 = np.concatenate(logit1_2, axis=0)
    logit2_2 = np.concatenate(logit2_2, axis=0)

    labels = logit3_to_label((logit0_2, logit1_2, logit2_2))

read image
image shape (3, 137, 236)
(3, 137, 236)
(3, 137, 236, 1)
v1_0_11
BengaliClassifier_v2(
  (encoder): SENetEncoder_CalibMixup_Multiscale_v2(
    (layer0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): Ba


gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': -10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': 10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
get_opt_bias_v1_0_11
read image
image shape (3, 137, 236)
{'degree': 0.0, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': -10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit


...


gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': 10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
get_opt_bias_v1_0_11
read image
image shape (3, 137, 236)
{'degree': 0.0, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': -10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
{'degree': 10, 'scale': 0.0, 'shift': [0.0, 0.0]}
predict logit



gra (3, 168)
vow (3, 11)
con (3, 7)
get_opt_bias_v1_0_11


In [27]:
print(logit0)
print()
print(logit0_2)

[[-0.33012036 -1.35116682  2.94207019 ...  0.07372455 -1.44254539
   1.23808107]
 [-0.42114628 -0.8974056  -1.13550346 ... -0.29479858 -1.02673934
   1.44556699]
 [-0.70218605 -1.1269382  -1.30650692 ...  0.99954567 -0.20243967
   2.22473667]
 ...
 [-0.48920596 -1.54734156 -1.3386622  ...  0.60619579 -0.2552648
   0.93145964]
 [-0.94645274 -1.97648153  0.88216288 ...  0.45498095 -0.60328022
   2.29666135]
 [-0.91028357 -2.14949737  0.02563278 ...  0.15252004 -0.97308884
   1.92199895]]

[[ 0.69451166  0.60192695  3.52676644 ...  0.49872175  0.32570992
   0.87222416]
 [ 0.91174431  1.18428704 -1.22945188 ...  0.18601426  0.66119703
   1.12085635]
 [ 1.18349154  1.42316356 -0.62222318 ...  1.00189772  0.61367914
   0.74048109]
 ...
 [ 0.6671722   0.76158606 -0.47000549 ...  0.2723898   0.2506436
   0.38408328]
 [ 0.12917538  0.5545364   0.93978097 ...  0.43411359  0.47024019
   0.85222406]
 [ 0.15853361  0.50048556 -0.160458   ...  0.2734234   0.04497945
   1.78604002]]


In [28]:
from scipy.special import softmax

a = 0.5
b = 0.5
labels = logit3_to_label((softmax(logit0,axis=1)*a + softmax(logit0_2,axis=1)*b, softmax(logit1,axis=1)*a + softmax(logit1_2,axis=1)*b, softmax(logit2,axis=1)*a + softmax(logit2_2,axis=1)*b))

In [29]:
print(softmax(logit0,axis=1))

[[1.56273979e-04 5.62926671e-05 4.12079463e-03 ... 2.34031483e-04
  5.13767487e-05 7.49804253e-04]
 [2.79322643e-05 1.73487962e-05 1.36730298e-05 ... 3.16940873e-05
  1.52440519e-05 1.80638124e-04]
 [5.59487254e-05 3.65866531e-05 3.05729240e-05 ... 3.06791198e-04
  9.22204616e-05 1.04456768e-03]
 ...
 [7.23559929e-05 2.51149348e-05 3.09428527e-05 ... 2.16372191e-04
  9.14268725e-05 2.99544769e-04]
 [1.51071622e-04 5.39320678e-05 9.40460512e-04 ... 6.13504582e-04
  2.12922140e-04 3.86945186e-03]
 [3.85714749e-05 1.11707551e-05 9.83398529e-05 ... 1.11644154e-04
  3.62234878e-05 6.55105005e-04]]


In [30]:
make_submission(labels)

make submission
num image  12
                       row_id  target
0  Test_0_consonant_diacritic       0
1        Test_0_grapheme_root       3
2      Test_0_vowel_diacritic       0
3  Test_1_consonant_diacritic       0
4        Test_1_grapheme_root      93
